### Create Linear Data With 3 Variables

In [1]:
import numpy as np
def norm_dist(mean, variance, size=1):
    if isinstance(size, int):
        size = size, # convert to tuple
    return mean + np.sqrt(variance) * np.random.randn(*size)
N = 500
np.random.seed(5)
## c_ concatenates columns of given arrays
## 1D array of size N have 1 columns and N rows

# independent variables
X = np.c_[norm_dist(22, 4, N),
      norm_dist(13, 2, N),
      norm_dist(-3, 3, N)]
# constants
C = norm_dist(5, 1, N)
# random error
err = norm_dist(0, 1, N)
# linear parameters
params = np.array([.1, .3, .6])

y = np.dot(X, params) + C + err
y[:5]

array([10.10989067, 11.50080798,  6.64404432,  9.24792969,  9.06743422])

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
X = sm.add_constant(X) ## add intercept (constant) terms on data
X[:5]

array([[ 1.        , 22.88245497, 14.50239519, -2.05989647],
       [ 1.        , 21.3382597 , 13.66963723, -2.30559953],
       [ 1.        , 26.86154237, 14.18029963, -1.75343357],
       [ 1.        , 21.49581574, 12.92343267, -3.03742079],
       [ 1.        , 22.21921968, 12.96374001, -2.95836168]])

### Ordinary Least Square (OLS) Regression

In [3]:
model=sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     91.39
Date:                Tue, 16 Apr 2019   Prob (F-statistic):           4.30e-47
Time:                        20:07:54   Log-Likelihood:                -874.02
No. Observations:                 500   AIC:                             1756.
Df Residuals:                     496   BIC:                             1773.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.9887      0.921      5.416      0.000       3.179       6.798
x1             0.0709      0.032      2.225      0.027       0.008       0.133
x2             0.3225      0.044      7.305      0.000       0.236       0.409
x3             0.5329      0.037     14.587      0.000       0.461       0.605
==============================================================================
Omnibus:                        0.401   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.818   Jarque-Bera (JB):                0.246
Skew:                          -0.022   Prob(JB):                        0.884
Kurtosis:                       3.099   Cond. No.                         382.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
results.params

array([4.98865928, 0.07086832, 0.32253465, 0.53287688])

### OLS Using DataFrame

In [5]:
import pandas as pd
data = pd.DataFrame(X, columns="const,var1,var2,var3".split(","))
data["y"] = y
results = smf.ols("y~var1+var2+var3", data).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     91.39
Date:                Tue, 16 Apr 2019   Prob (F-statistic):           4.30e-47
Time:                        20:07:54   Log-Likelihood:                -874.02
No. Observations:                 500   AIC:                             1756.
Df Residuals:                     496   BIC:                             1773.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.9887      0.921      5.416      0.000       3.179       6.798
var1           0.0709      0.032      2.225      0.027       0.008       0.133
var2           0.3225      0.044      7.305      0.000       0.236       0.409
var3           0.5329      0.037     14.587      0.000       0.461       0.605
==============================================================================
Omnibus:                        0.401   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.818   Jarque-Bera (JB):                0.246
Skew:                          -0.022   Prob(JB):                        0.884
Kurtosis:                       3.099   Cond. No.                         382.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
results.predict(data[:5])

0    10.190154
1     9.681197
2    10.531566
3     9.061715
4     9.168111
dtype: float64

In [7]:
y[:5]

array([10.10989067, 11.50080798,  6.64404432,  9.24792969,  9.06743422])

### Autoregressive Time Series 

In [13]:
init_x = 4
time_series = np.array([init_x, init_x])
lag_coff = [0.4, 0.11]
N = 100
noise = norm_dist(mean=0, variance=0.1, size=N)
for i in range(N):
    next_value = time_series[-1] * lag_coff[-1] + time_series[-2] * lag_coff[-2] + noise[i]
    # next_value depends on previous two values
    time_series = np.append(time_series, next_value)

time_series

array([ 4.        ,  4.        ,  1.8341935 ,  1.48961368,  1.02934339,
        0.77503669,  0.46211241,  0.38831705,  0.01156281,  0.24286372,
       -0.1724352 ,  0.46832851,  0.39871877,  0.3559631 , -0.10822737,
       -0.19335087,  0.00723531,  0.3819059 , -0.12793255,  0.18843265,
        0.34245406, -0.21671975,  0.42853952,  0.44600636,  0.18784942,
       -0.09243029, -0.30747568,  0.13384781, -0.20235526,  0.44526582,
        0.35563615,  0.46818134,  0.2981005 ,  0.29169219,  0.32306306,
        0.76421547, -0.08691831,  0.67214685, -0.0688    ,  0.48049011,
       -0.41058494,  0.57529644, -0.35711183,  0.11009298, -0.58079554,
       -0.07454406, -0.47634739, -0.07914306,  0.30499578,  0.08900433,
       -0.42255844, -0.8546123 , -0.31439332, -0.82664626,  0.01402777,
       -0.43502386,  0.07879263,  0.30660912,  0.37569716,  0.11343556,
       -0.06465325,  0.50350115,  0.09549889,  0.41426538,  0.13782285,
        0.58286818,  0.16667549,  0.18494218,  0.47115045,  0.16

In [17]:
## for real world data we can't know number of lag terms so we guess that slightly high
MAX_LAGS = 5
model = sm.tsa.AR(time_series)
results = model.fit(MAX_LAGS)

In [20]:
results.params

array([ 0.074626  ,  0.14619054,  0.48062425, -0.04383747,  0.04807169,
       -0.06327487])